### Required Functions

Functions and methods required for notebook calculations

In [3]:
def CalcN_0(par, n, t, rate, src, vol, tt=0.0):
    """
    Calculates the initial population of atoms post-irradiation  
   
    Parameters
    ==========
    par : PyNE nucname string
        Id of the parent isotope
    n: int
        Initial number of parent atoms from previous irradiations
    t: int
        Irradiation time in sec
    rate : float
        The reaction rate in rx per sec per src per cm3
    src : float
        The source strength in n per sec    
    vol : float
        The volume of the foil in cm3  
    tt: int
        Post-irradiation transfer time in sec

    Optional
    ========
        
    Returns
    =======
    N_0 float
        The number of atoms after irradiation time t 
    """ 
    
    if par=="In116M":
        N_0=rate*vol*src/2.12792E-4*(1-m.exp(-2.12792E-4*t))+n*m.exp(-2.12792E-4*t)
        N_0=N_0*m.exp(-2.12792E-4**tt)
    else:
        N_0=rate*vol*src/data.decay_const(par)*(1-m.exp(-data.decay_const(par)*t))+n*m.exp(-data.decay_const(par)*t)
        N_0=N_0*m.exp(-data.decay_const(par)*tt)
    
    return N_0

#-------------------------------------------------------------------------------------------------------------#
def Activity(t, par, n):
    """
    Calculates the activity of a given isotope at time t after the irradiation.  
   
    Parameters
    ==========
    t: int
        Time post irradiation
    par : PyNE nucname string
        Id of the parent isotope
    n: int
        Initial number of parent atoms

    Optional
    ========
        
    Returns
    =======
    act : float
        The activity at time t in decays/s
    """ 
    
    if par=="In116M":
        act=2.12792E-4*n*m.exp(-2.12792E-4*t)
    else:
        act=data.decay_const(par)*n*m.exp(-data.decay_const(par)*t)
    
    return act

#-------------------------------------------------------------------------------------------------------------#
def GCF(r_src, r_det, det2src):
    """
    Calculates the solid angle geometry correction factor for the detector configuration from Knoll p. 119  
   
    Parameters
    ==========
    r_det : float
        Radius of the detector in cm
    r_src: float
        Radius of the foil in cm
    det2src: float
        Distance from the detector to src in cm 

    Optional
    ========
        
    Returns
    =======
    gcf : float
        The gcf for the given configuration
    """ 

    alpha=(r_src/det2src)**2
    beta=(r_det/det2src)**2
    f1=5./16.*(beta/(1+beta)**(7./2.))-35./64.*(beta**2/(1+beta)**(9./2.))
    f2=35./128.*(beta/(1+beta)**(9./2.))-315./256.*(beta**2/(1+beta)**(11./2.))+1155./1028.*(beta**3/(1+beta)**(13./2.))
    gcf=0.5*(1-1./(1+beta)**(1./2.)-3./8.*(alpha*beta/(1+beta)**(5./2.))+alpha**2*f1-alpha**3*f2)

    return gcf

#-------------------------------------------------------------------------------------------------------------#
def CalcRelEff(e,det2src):
    """
    Calculates the relative efficiency based on calibration curves at a given distance  
   
    Parameters
    ==========
    e : float
        Incident gamma ray energy in keV
    det2src: float
        Distance from the detector to src in cm 

    Optional
    ========
        
    Returns
    =======
    eff : float
        The relative efficienty for the given configuration and line
    """ 
    
    if det2src==0:
        eff=10**(0.6368*10-0.1175*10*m.log10(e)+0.9785*0.1*m.log10(e)**2-(0.3278*10**4)/(e**2))
    elif det2src==1:
        eff=10**(0.5166*10-0.4065*m.log10(e)-0.4308*0.1*m.log10(e)**2-0.1888*10**4/e**2)
    elif det2src==2:
        eff=10**(0.5722*10-0.8004*m.log10(e)+0.1418*0.1*m.log10(e)**2-0.2112*10**4/e**2)
    elif det2src==3:
        eff=10**(0.585*10-0.9388*m.log10(e)+0.3163*0.1*m.log10(e)**2-0.2266*10**4/e**2)
    elif det2src==4:
        eff=10**(0.5736*10-0.9279*m.log10(e)+0.2328*0.1*m.log10(e)**2-0.2373*10**4/e**2)
    elif det2src==5:
        eff=10**(0.6114*10-0.1225*10*m.log10(e)+0.6998*0.1*m.log10(e)**2-0.2768*10**4/e**2)
    else:
        print "No calibration exists for that detector to source distance"

    return eff

# Activation Counting Tools

This notebook is to aid in determining optimal count times given a set of activations and experiments design parameters. It requires PyNE to be loaded for nuclear data purposes.

Import the following packages:

In [4]:
import numpy as np

from pyne import data
from pyne import nucname
from scipy.integrate import quad


# Path to support scripts 
sys.path.insert(0,os.path.abspath('/home/pyne-user/Dropbox/UCB/Computational_Tools/Scripts/Python/Support'))
from BasicNuclearCalcs import FractionalSolidAngle, Activity, N_0
from DetectorCals import volumeGCF, CalcRelEff

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:4: QAWarning: pyne.data is not yet QA compliant.


## User Experimental Input

Specify the experimental parameters determining the reaction rate.  This assumes that the T<sub>1/2</sub> is >> 1 seconds, which should be true for all practical experiments.

### Beam Variables
src = the neutron source strength in in n/sec

src_t = irradition time in s

In [5]:
src=9944835105
t=57600

### Foil Variables
foil = name of the foil reaction product written in the format "XXAAA" - ex "U235" or "Rb86"

rx_rate = the reaction rate per source particle in units of reactions/cm<sup>3</sup>/src (this can be obtained from simulation or a simple calculation). It assumes that the natural abundance of the isotope is accounted for in this rate.  

foil_r = the foil radius in cm

foil_h = the foil height in cm

foil_rho = the foil density in g cm<sup>-3</sup>

vol = the foil volume in cm<sup>3</sup>

trans_t = the foil transfer time post-irradiation in sec

In [6]:
#Basic foil parameters
act_product="Co58"
rx_rate=2.66E-6
foil_r=2.5
foil_h=0.1
foil_rho=8.908
BR=99.45
gamma_energy=810.76

#Delay between irradiation and counting
decay_t=0

# Calculate the foil volume
vol=np.pi*foil_r**2*foil_h

# Calculate the initial number of atoms from the irradiation (assumes natural abundance is captured in rx_rate)
N_0=CalcN_0(act_product, 0.0, t, rx_rate, src, vol, decay_t)

print "The number of {} atoms in the sample after irradiation and {} s decay= {:.3e}".format(act_product,decay_t,N_0)

The number of Co58 atoms in the sample after irradiation and 0 s decay= 2.982e+09


### Decay Data

The following uses PyNE to get decay data for the foil.  This is used to get the branching ratio of the primary gamma and provide general information about the decay radiation.

Future revision can use this data to add automated functionality such as background count given the primary gamma.  

In [7]:
# Get level transitions
decay_pairs = data.gamma_from_to_byparent(nucname.id(act_product))

# Get gamma ray energies
energies = data.gamma_energy(nucname.id(act_product))

# Relative gamma intensities
intensities = data.gamma_photon_intensity(nucname.id(act_product))

# Converts the relative intensities to decays per 100 decays of the parent and print 
print "Parent (t_1/2 [s])    Daughter    Level Transition     Gamma (keV)           BR (%)"
print "======================================================================================="
photonbr, photonbr_error = data.decay_photon_branch_ratio(nucname.id(act_product),decay_pairs[0][1])
final_intensities = []
for i in range(len(intensities)):
    # compute the intensities by multiplying the branch ratio and the relative intensity; ignore the errors 
    final_intensities.append(photonbr*intensities[i][0])
    print "{} ({})        {:5s}           {}->{}           {} +/-{}         {}".format(act_product, data.half_life(act_product), 
                                            nucname.name(decay_pairs[i][1]), decay_pairs[i][0]%100, \
                                            decay_pairs[i][1]%100, energies[i][0], energies[i][1],\
                                            final_intensities[-1])
    
print "\n\n\nWARNING:BR and gamma energy must be specified by user. PyNE automated interfaces removed due to incomplete data."

Parent (t_1/2 [s])    Daughter    Level Transition     Gamma (keV)           BR (%)
Co58 (6122304.0)        Fe58            1->0           810.7593 +/-0.002         99.45
Co58 (6122304.0)        Fe58M           2->1           863.951 +/-0.006         0.686205
Co58 (6122304.0)        Fe58            2->0           1674.725 +/-0.007         0.51714





### Counting Facility Variables
background = the background rate at the peak of interest in counts/s

det_r = radius of the detector in cm

det_foil_dist = the distance from the detector face to the foil in cm

eff = the detection efficiency for a given gamma

sigma = the desired counting statistics level

In [10]:
background=0.01
det_r=5.08
det2foil_dist=1
sigma=0.01

### Calculate the absolute efficiency

Takes into account the geometry correction factor (gcf), detector intrinsic efficiency, and the energy and position dependence of those variables. Change the foil distance if significant dead time encountered.

In [11]:
while True:
    # Calculate the GCF
    gcf=GCF(foil_r,det_r,det2foil_dist)

    # Calculate absolute efficiency
    abs_cf=[(0,95775.80,4.7077),(1,6016.50,2.6323),(2,5694.63,1.952),(3,4392.66,1.4806),(4,3172.02,1.0245),(5,2643.60,0.8170)]
    ref_rel_eff=abs_cf[det2foil_dist][1]
    rel_eff=CalcRelEff(gamma_energy,det2foil_dist)
    abs_eff=rel_eff/ref_rel_eff*abs_cf[det2foil_dist][2]
    
    print rel_eff, ref_rel_eff, abs_cf[det2foil_dist][2]

    # Use a simple paralyzable model to calculate worst case scenario
    n=Activity(0,act_product,N_0)*abs_eff/100*BR/100
    tau=1E-5
    meas=n*m.exp(-n*tau)

    if n/meas>1.01:
        det2foil_dist+=1
    else:
        break
        
    if det2foil_dist>5:
        print "ERROR: The foil is hot!"
        break

print "The foil is {} cm from the detector, and the dead time is {}%".format(det2foil_dist,(n/meas-1)*100)
print "The GCF is {}, and the overall absolute efficiency is {:2f}% @ {} keV".format(gcf,abs_eff,gamma_energy)

4131.7589925 6016.5 2.6323
The foil is 1 cm from the detector, and the dead time is 0.00606975650512%
The GCF is 0.394064814077, and the overall absolute efficiency is 1.807700% @ 810.76 keV


### Counting time calculations

Bringing it all together....

In [46]:
# Define the activity integrand accounting for all of the efficiencies
def integrand(t):
    return Activity(t,act_product,N_0)*abs_eff/100*BR/100
    
# Report the starting activities
print "The activity of {} in the sample at the start of counting ({} s after irradiation) = {}"\
       .format(act_product,decay_t,Activity(0,act_product,N_0)*BR/100)
print "The specific activity of {} in the sample at the start of counting ({} s after irradiation) = {}"\
       .format(act_product,decay_t,Activity(0,act_product,N_0)*BR/100/(vol*foil_rho))

# Approximate the optimal foil counting time using an average count rate
tf=1
diff=1000
try:
    while diff > 1:
        prevt=tf
        S = quad(integrand, 0, tf)[0]/tf
        tf=((m.sqrt(S+background)+m.sqrt(background))**2/(sigma**2*S**2))/(1+1/m.sqrt((S+background)/background))  #Knoll eqn 3.54/55
        diff=tf-prevt
    print "The integrated count rate in the detector over the counting period is {:.3f}".format(S)
    print "The optimal count time for the {} foil with a desired statistical level of sigma={} is {:.0f} sec [{:.2f} hr]."\
          .format(act_product,sigma,tf,tf/3600)
    # Calculate the optimal time for background counting
    tb=tf/m.sqrt((S+background)/background)
    print "The optimal time for background counting is {:.0f} sec [{:.2f} hr]".format(tb,tb/3600)
except ZeroDivisionError:
    tf=1E99
    print "The desired level of statistics cannot be achieved with this setup"


The activity of Co58 in the sample at the start of counting (0 s after irradiation) = 335.762079176
The specific activity of Co58 in the sample at the start of counting (0 s after irradiation) = 19.1964776304
The integrated count rate in the detector over the counting period is 6.069
The optimal count time for the Co58 foil with a desired statistical level of sigma=0.01 is 1717 sec [0.48 hr].
The optimal time for background counting is 70 sec [0.02 hr]


In [50]:
print S, background, sigma
print ((m.sqrt(S+background)+m.sqrt(background))**2/(sigma**2*S**2))/(1+1/m.sqrt((S+background)/background))

6.06898232843 0.01 0.01
1717.37738828
